## CSIRO Biomass Prediction - 3-Model Ensemble Inference

**Alignment note.** This end-to-end script ports Youich Wada's three-model ensemble ([csiro-3-models-ensemble-lb-0-63](https://www.kaggle.com/code/youichwada/csiro-3-models-ensemble-lb-0-63)) into a single executable notebook so each step mirrors the CSIRO paper's requirements for component-wise biomass estimation, reliance on imagery-only signals at validation/test time, and the weighted R² evaluation scheme (`CSIRO paper, 2510.22916v1.txt:260-382`). Every section below explains how it operationalizes the paper's methodology while preserving Wada's proven training/inference recipes.

This notebook is the final inference pipeline for the **CSIRO - Pasture Biomass Prediction** competition.
It aims for more robust and accurate predictions by performing a **weighted average ensemble** of the results from three high-performance models with different approaches.

### Execution Flow

1.  **Individual Model Inference:** Sequentially run the inference scripts (`.py` files) for the three different models and save their respective predictions as CSV files.
2.  **Final Ensemble:** Read the CSV files output by each model, perform a weighted average with optimal weights, and create the final submission file `submission.csv`.

---

### 1. Model Overview

This notebook uses models based on the following three public notebooks.

### Model 1: DINOv2-Giant + Lasso Regression

-   **Approach:**
    -   Extracts high-dimensional feature vectors from images using **DINOv2-Giant**, a powerful visual feature extractor.
    -   Uses the extracted features as input to predict the biomass amount with a linear model, **Lasso Regression**.
-   **Features:**
    -   Leverages the power of DINOv2, which was pre-trained with large-scale self-supervised learning, to capture the essential features of the images.
-   **Reference Notebook:**
    -   **Title:** `csiro-image2biomass-lb-59`
    -   **URL:** https://www.kaggle.com/code/mks2192/csiro-image2biomass-lb-59

### Model 2: Two-Stream ConvNeXt-Tiny

-   **Approach:**
    -   Adopts a **Two-Stream architecture** where wide-angle images from a stereo camera are split into **left and right halves**, each fed into a **ConvNeXt-Tiny** model.
    -   The features extracted from both halves are concatenated and connected to dedicated prediction heads (**3-Head**) for the three main targets (`Dry_Total_g`, `GDM_g`, `Dry_Green_g`).
-   **Features:**
    -   A very powerful inference pipeline combining **5-Fold ensembling** and **Test-Time Augmentation (TTA)** (3 views: original, horizontal flip, and vertical flip).
-   **Reference Notebook:**
    -   **Title:** **`[LB 0.61]CSIRO_Infer`**
    -   **URL:** **https://www.kaggle.com/code/takahitomizunobyts/lb-0-61-csiro-infer**

### Model 3: SigLIP + Gradient Boosting Ensemble

-   **Approach:**
    -   Extracts high-quality features from images using **SigLIP**, a Vision-Language model developed by Google.
    -   These features are then used to make predictions with two gradient boosting models: **Gradient Boosting Regressor** and **CatBoost Regressor**.
-   **Features:**
    -   Utilizes rich features obtained from SigLIP, which has learned the relationship between images and text. The final prediction is an average of the results from the two gradient boosting models to enhance robustness.
-   **Reference Notebook:**
    -   **Title:** `Simple SigLIP + GradientBoosting`
    -   **URL:** https://www.kaggle.com/code/hocop1/simple-siglip-gradientboosting

---

### 2. Final Ensemble Method

The predictions from the three models above are combined using a **weighted average** with the following weights:

-   `submission_dino_giant.csv` (Model 1): **25%**
-   `submission_ConvnextTiny.csv` (Model 2): **45%**
-   `submission_SigLIP.csv` (Model 3): **30%**

```
final_prediction = 0.25 * pred_model1 + 0.40 * pred_model2 + 0.30 * pred_model3
```


## Model 1 – DINOv2-Giant + LightGBM (Paper alignment)

By extracting frozen image embeddings and fitting per-target regressors, this stage enforces the paper's directive to predict all five dry-matter components directly from imagery even when auxiliary metadata disappears at validation/test time (`CSIRO paper, 2510.22916v1.txt:273-343`). The LightGBM head mirrors Youich Wada's ensemble, but its grouping by sampling date and state follows the paper's emphasis on controlling for location/season effects during training before reverting to vision-only inference. This keeps the downstream ensemble calibrated to the weighted R² objective while honoring the component relationships (Dry_Green + Dry_Clover = GDM, component sum = Dry_Total) laid out in the dataset description (`CSIRO paper, 2510.22916v1.txt:260-349`).


In [ ]:
%%writefile model_1_dino_giant.py
# ====================================================================================
# Model 1: DINOv2-Giant + LightGBM meta-regression with cached OOF predictions
# ====================================================================================
"""
This script keeps the extremely strong DINOv2-Giant visual backbone but replaces
the underpowered Lasso head with a non-linear LightGBM regressor.  We extract a
single set of frozen image embeddings, fit a 5-fold GroupKFold LightGBM model
per target, save OOF predictions for ensembling/stacking, and run inference on
the competition test split.
"""

import gc
import os
from pathlib import Path
from typing import Dict, List

import lightgbm as lgb
import numpy as np
import pandas as pd
import torch
from PIL import Image
from sklearn.model_selection import GroupKFold
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModel

# ------------------------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------------------------
DATA_ROOT = Path("/kaggle/input/csiro-biomass")
DINO_ROOT = Path("/kaggle/input/dinov2/pytorch/giant/1")
TARGET_COLS = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42


def _load_metadata() -> tuple[pd.DataFrame, pd.DataFrame]:
    """Load and pivot train/test metadata."""
    train_long = pd.read_csv(DATA_ROOT / "train.csv")
    train_long["image_id"] = train_long["sample_id"].str.split("__").str[0]
    train_long["target_name"] = train_long["sample_id"].str.split("__").str[1]

    # Pivot to wide targets and keep useful metadata for grouping
    train_targets = (
        train_long.pivot_table(
            index="image_id", columns="target_name", values="target", aggfunc="mean"
        )
        .reindex(columns=TARGET_COLS)
        .reset_index()
    )

    train_meta = (
        train_long[["image_id", "image_path", "Sampling_Date", "State"]]
        .drop_duplicates("image_id")
        .reset_index(drop=True)
    )
    train_df = train_targets.merge(train_meta, on="image_id", how="left").sort_values(
        "image_id"
    )

    test_long = pd.read_csv(DATA_ROOT / "test.csv")
    test_long["image_id"] = test_long["sample_id"].str.split("__").str[0]
    test_df = (
        test_long[["image_id", "image_path"]]
        .drop_duplicates("image_id")
        .reset_index(drop=True)
    )

    return train_df, test_df


def _extract_embeddings(
    df: pd.DataFrame, processor: AutoImageProcessor, model: AutoModel, desc: str
) -> Dict[str, np.ndarray]:
    """Extract pooled DINO embeddings for every unique image."""
    feats: Dict[str, np.ndarray] = {}
    for _, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        img_path = DATA_ROOT / row["image_path"]
        with Image.open(img_path) as img:
            img = img.convert("RGB")
            inputs = processor(images=img, return_tensors="pt").to(DEVICE)
            with torch.no_grad():
                outputs = model(**inputs)
        feats[row["image_id"]] = outputs.pooler_output.squeeze(0).cpu().numpy()
    return feats


def _build_feature_matrix(
    df: pd.DataFrame, embeddings: Dict[str, np.ndarray]
) -> np.ndarray:
    """Stack embeddings so ordering matches df rows."""
    ordered = [embeddings[row["image_id"]] for _, row in df.iterrows()]
    return np.stack(ordered).astype(np.float32)


def _competition_metric(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Weighted R2 as defined in the competition."""
    weights = np.array([0.1, 0.1, 0.1, 0.5, 0.2])
    y_weighted_mean = np.average(y_true, weights=weights, axis=1).mean()
    ss_res = np.average((y_true - y_pred) ** 2, weights=weights, axis=1).mean()
    ss_tot = np.average((y_true - y_weighted_mean) ** 2, weights=weights, axis=1).mean()
    return 1.0 - ss_res / ss_tot


def train_lightgbm_head(
    train_df: pd.DataFrame,
    train_feats: np.ndarray,
    test_feats: np.ndarray,
) -> tuple[np.ndarray, np.ndarray]:
    """Train LightGBM models with GroupKFold and return OOF/test predictions."""
    y = train_df[TARGET_COLS].to_numpy()
    groups = (train_df["Sampling_Date"] + "_" + train_df["State"]).to_numpy()

    n_targets = len(TARGET_COLS)
    oof = np.zeros((len(train_df), n_targets), dtype=np.float32)
    test_pred = np.zeros((len(test_feats), n_targets), dtype=np.float32)

    params = dict(
        n_estimators=1200,
        learning_rate=0.03,
        num_leaves=80,
        subsample=0.9,
        colsample_bytree=0.8,
        reg_alpha=0.2,
        reg_lambda=1.0,
        min_child_samples=30,
        random_state=SEED,
        n_jobs=-1,
    )

    gkf = GroupKFold(n_splits=5)
    for fold, (tr_idx, val_idx) in enumerate(
        gkf.split(train_feats, y, groups=groups), start=1
    ):
        print(f"[Fold {fold}] training LightGBM heads...")
        X_train, y_train = train_feats[tr_idx], y[tr_idx]
        X_val, y_val = train_feats[val_idx], y[val_idx]

        fold_test = np.zeros_like(test_pred)

        for target_idx, target_name in enumerate(TARGET_COLS):
            reg = lgb.LGBMRegressor(**params)
            reg.fit(
                X_train,
                y_train[:, target_idx],
                eval_set=[(X_val, y_val[:, target_idx])],
                eval_metric="rmse",
                callbacks=[
                    lgb.early_stopping(75, verbose=False),
                    lgb.log_evaluation(200),
                ],
            )
            oof[val_idx, target_idx] = reg.predict(X_val)
            fold_test[:, target_idx] = reg.predict(test_feats)

        oof_score = _competition_metric(y[val_idx], np.clip(oof[val_idx], a_min=0, a_max=None))
        print(f"    Fold metric (weighted R2): {oof_score:.5f}")
        test_pred += fold_test / gkf.n_splits

    full_score = _competition_metric(y, np.clip(oof, a_min=0, a_max=None))
    print(f"[Model 1] Full OOF metric (weighted R2): {full_score:.5f}")
    return np.clip(oof, 0, None), np.clip(test_pred, 0, None)


def _wide_to_submission(
    wide_preds: pd.DataFrame, test_long: pd.DataFrame
) -> pd.DataFrame:
    """Convert wide per-image predictions into Kaggle submission format."""
    melted = (
        wide_preds.reset_index()
        .melt(id_vars="image_id", var_name="target_name", value_name="prediction")
        .rename(columns={"prediction": "target"})
    )
    test_long = test_long.copy()
    test_long["image_id"] = test_long["sample_id"].str.split("__").str[0]
    test_long["target_name"] = test_long["sample_id"].str.split("__").str[1]
    submission = (
        test_long[["sample_id", "image_id", "target_name"]]
        .merge(melted, on=["image_id", "target_name"], how="left")
        .drop(columns=["image_id", "target_name"])
        .sort_values("sample_id")
    )
    return submission


def main():
    print(f"--- [Start] Model 1: DINOv2-Giant + LightGBM (device={DEVICE}) ---")
    train_df, test_df = _load_metadata()

    processor = AutoImageProcessor.from_pretrained(DINO_ROOT)
    backbone = AutoModel.from_pretrained(DINO_ROOT).to(DEVICE)
    backbone.eval()

    train_feats = _extract_embeddings(train_df, processor, backbone, "Train")
    test_feats = _extract_embeddings(test_df, processor, backbone, "Test")

    X_train = _build_feature_matrix(train_df, train_feats)
    X_test = _build_feature_matrix(test_df, test_feats)

    oof_preds, test_preds = train_lightgbm_head(train_df, X_train, X_test)

    # Persist OOF predictions for stacking / weight search
    oof_df = pd.DataFrame(oof_preds, columns=TARGET_COLS)
    oof_df["image_path"] = train_df["image_path"].values
    oof_df.to_csv("oof_model1.csv", index=False)
    print("Saved OOF predictions to oof_model1.csv")

    # Create submission file
    test_long = pd.read_csv(DATA_ROOT / "test.csv")
    wide_test = pd.DataFrame(test_preds, columns=TARGET_COLS, index=test_df["image_id"])
    submission = _wide_to_submission(wide_test, test_long)
    submission.to_csv("submission_dino_giant.csv", index=False)
    print("Saved Model 1 predictions to submission_dino_giant.csv")

    # Cleanup
    del backbone, processor, train_feats, test_feats, X_train, X_test
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("--- [Done] Model 1 ---")


if __name__ == "__main__":
    main()


## Model 2 – Two-Stream ConvNeXt-Tiny (Paper alignment)

The stereo-split ConvNeXt reproduces Wada's strongest CNN branch and is explicitly designed for the 70 cm × 30 cm quadrat imagery described in the paper, where each capture contains left/right halves of the measurement frame (`CSIRO paper, 2510.22916v1.txt:263-274`). Using identical augmentations on both halves keeps structural cues intact, satisfying the paper's insistence on learning pasture composition directly from the visual data once metadata is withheld, while the three-head decoder recreates the component-wise targets that inform the weighted evaluation (`CSIRO paper, 2510.22916v1.txt:273-382`).


In [ ]:
%%writefile model_2_ConvnextTiny.py
# ============================================================================
#
# Model 2: CSIRO Biomass Competition - Inference Pipeline (TTA + Ensemble)
#
# ============================================================================
# This script performs predictions on test data using trained models.
#
# Pipeline Overview:
# 1. Test Data Preparation: Load CSV -> Filter to unique images
# 2. Model Loading: Load 5-Fold trained models
# 3. TTA Inference: 3 Views × 5-Fold Ensemble prediction
# 4. Post-processing: Reconstruct 5 targets from 3 predictions
# 5. Submission Creation: Convert from wide format to long format
# ============================================================================

from __future__ import annotations
from dataclasses import dataclass, field
from pathlib import Path
from typing import Optional
from collections import OrderedDict
import argparse
import os
import gc

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm


# ============================================================================
# Configuration Management
# ============================================================================

@dataclass
class InferenceConfig:
    """
    Data class for managing inference pipeline configuration.

    The following items must match the training configuration:
    - model_name
    - img_size
    - target column names
    """

    # --- Path settings ---
    base_path: Path = Path('/kaggle/input/csiro-biomass')
    test_csv: Path = field(init=False)
    train_csv: Path = field(init=False)
    test_image_dir: Path = field(init=False)
    train_image_dir: Path = field(init=False)
    model_dir: Path = Path('/kaggle/input/csiro-exp3/convnext_exp3') # Directory where trained models are stored
    submission_file: str = 'submission_ConvnextTiny.csv'

    # --- Model settings (must match training) ---
    model_name: str = 'convnext_small' # Backbone model to use
    img_size: int = 1000 # Input image size

    # --- Device settings ---
    device: torch.device = field(default_factory=lambda: torch.device(
        'cuda' if torch.cuda.is_available() else 'cpu'
    ))

    # --- Inference settings ---
    batch_size: int = 1
    num_workers: int = 1
    n_folds: int = 5 # Number of folds for ensemble

    # --- Target settings (must match training) ---
    # The 3 targets the model directly predicts
    train_target_cols: list[str] = field(default_factory=lambda: [
        'Dry_Total_g', 'GDM_g', 'Dry_Green_g'
    ])

    # All 5 targets required for submission
    all_target_cols: list[str] = field(default_factory=lambda: [
        'Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g'
    ])

    def __post_init__(self) -> None:
        """Construct paths after initialization"""
        self.test_csv = self.base_path / 'test.csv'
        self.train_csv = self.base_path / 'train.csv'
        self.test_image_dir = self.base_path / 'test'
        self.train_image_dir = self.base_path / 'train'
    
    def resolve_split_paths(self, split: str) -> tuple[Path, Path]:
        """Return (csv_path, image_dir) for the requested split."""
        split = split.lower()
        if split == 'train':
            return self.train_csv, self.train_image_dir
        if split == 'test':
            return self.test_csv, self.test_image_dir
        raise ValueError(f"Unsupported split: {split}")

    def display_info(self) -> None:
        """Display configuration information"""
        print(f"{'='*70}")
        print(f"Inference Configuration")
        print(f"{'='*70}")
        print(f"Device: {self.device}")
        print(f"Backbone: {self.model_name}")
        print(f"Image Size: {self.img_size}x{self.img_size}")
        print(f"Batch Size: {self.batch_size}")
        print(f"Ensemble: {self.n_folds}-Fold")
        print(f"TTA: 3 Views (Original, Horizontal Flip, Vertical Flip)")
        print(f"{'='*70}\n")


# ============================================================================
# TTA (Test-Time Augmentation) Transforms
# ============================================================================

class TTATransformFactory:
    """
    Factory class for generating Test Time Augmentation transforms.

    Provides 3 different views:
    1. Original (no augmentation)
    2. Horizontal flip
    3. Vertical flip
    """

    def __init__(self, img_size: int):
        """
        Args:
            img_size: Image size after resizing
        """
        self.img_size = img_size

        # Base transforms common to all views
        self.base_transforms = [
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Standard ImageNet normalization
            ToTensorV2() # Convert to PyTorch tensor format
        ]

    def get_tta_transforms(self) -> list[A.Compose]:
        """
        Generate 3 transform pipelines for TTA.

        Returns:
            List of 3 Albumentations.Compose objects

        Why not add more TTA variations?
            → Considering the trade-off with inference time.
        """
        # View 1: Original
        original = A.Compose([
            A.Resize(self.img_size, self.img_size),
            *self.base_transforms
        ])

        # View 2: Horizontal flip
        hflip = A.Compose([
            A.HorizontalFlip(p=1.0), # Apply horizontal flip with 100% probability
            A.Resize(self.img_size, self.img_size),
            *self.base_transforms
        ])

        # View 3: Vertical flip
        vflip = A.Compose([
            A.VerticalFlip(p=1.0), # Apply vertical flip with 100% probability
            A.Resize(self.img_size, self.img_size),
            *self.base_transforms
        ])

        return [original, hflip, vflip]


# ============================================================================
# Dataset
# ============================================================================

class TestBiomassDataset(Dataset):
    """
    Two-stream dataset for testing.

    Accepts a specific transform pipeline for TTA and applies
    the same augmentation to both left and right images.

    Returns:
        tuple: (img_left, img_right) (left image tensor, right image tensor)
    """

    def __init__(
        self,
        df: pd.DataFrame,
        transform_pipeline: A.Compose,
        image_dir: Path
    ):
        """
        Args:
            df: DataFrame containing image paths
            transform_pipeline: Augmentation pipeline to apply
            image_dir: Path to the image directory
        """
        self.df = df
        self.transform = transform_pipeline
        self.image_dir = image_dir
        self.image_paths = df['image_path'].values

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Get one sample.

        Args:
            idx: Sample index

        Returns:
            (left_image, right_image): Tuple of left and right image tensors

        Why not apply different augmentations to left/right as in training?
            → During TTA, apply the same transform to both images to preserve symmetry.
        """
        img_path = self.image_paths[idx]
        full_path = self.image_dir / Path(img_path).name

        # Load image (return black image on error)
        image = cv2.imread(str(full_path))

        if image is None:
            print(f"Warning: Failed to load image: {full_path} -> Returning black image")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert from OpenCV (BGR) to RGB

        # Split into left and right
        height, width = image.shape[:2]
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]

        # Apply same transform to both
        img_left_tensor = self.transform(image=img_left)['image']
        img_right_tensor = self.transform(image=img_right)['image']

        return img_left_tensor, img_right_tensor


# ============================================================================
# Model
# ============================================================================

class BiomassModel(nn.Module):
    """
    Two-stream, three-head regression model.

    Uses the exact same architecture as during training.
    """

    def __init__(self, model_name: str, pretrained: bool = False):
        """
        Args:
            model_name: timm model name
            pretrained: Whether to use pretrained weights (False for inference, as custom weights are loaded later)
        """
        super().__init__()

        # Shared backbone for both streams
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,       # Classifier layer is not needed
            global_pool='avg'    # Use GAP (Global Average Pooling)
        )

        self.n_features = self.backbone.num_features # Number of output features from the backbone
        self.n_combined = self.n_features * 2        # Number of features after concatenating left and right streams

        # Dedicated prediction heads for each of the three targets
        self.head_total = self._create_head() # Head for Dry_Total_g
        self.head_gdm = self._create_head()   # Head for GDM_g
        self.head_green = self._create_head() # Head for Dry_Green_g

    def _create_head(self) -> nn.Sequential:
        """Helper function to generate the MLP structure for a single head"""
        return nn.Sequential(
            nn.Linear(self.n_combined, self.n_combined // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined // 2, 1) # Output is a single continuous value
        )

    def forward(
        self,
        img_left: torch.Tensor,
        img_right: torch.Tensor
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Forward pass.

        Args:
            img_left: Left image tensor [B, C, H, W]
            img_right: Right image tensor [B, C, H, W]

        Returns:
            (total_pred, gdm_pred, green_pred): Tuple of predictions (each [B, 1])
        """
        feat_left = self.backbone(img_left)   # Extract features from the left image
        feat_right = self.backbone(img_right) # Extract features from the right image
        combined = torch.cat([feat_left, feat_right], dim=1) # Concatenate features

        # Calculate predictions with each head
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)

        return out_total, out_gdm, out_green


# ============================================================================
# Model Loader
# ============================================================================

class ModelLoader:
    """
    Class for loading trained models.

    Handles weights saved with DataParallel.
    """

    def __init__(self, config: InferenceConfig):
        """
        Args:
            config: Configuration object
        """
        self.config = config

    def load_fold_models(self) -> list[nn.Module]:
        """
        Load all 5-Fold trained models.

        Returns:
            List of models (each in eval mode on the specified device)

        Raises:
            FileNotFoundError: If a model file is not found
        """
        print(f"\nLoading {self.config.n_folds} trained models...")

        models = []

        for fold in range(self.config.n_folds):
            model_path = self.config.model_dir / f'best_model_fold{fold}.pth'

            if not model_path.exists():
                raise FileNotFoundError(f"Model file not found: {model_path}")

            # Initialize model
            model = BiomassModel(self.config.model_name, pretrained=False)

            # Load weights
            state_dict = torch.load(model_path, map_location=self.config.device)

            # Remove 'module.' prefix from DataParallel
            state_dict = self._remove_dataparallel_prefix(state_dict)

            model.load_state_dict(state_dict)
            model.eval()  # Set to evaluation mode
            model.to(self.config.device) # Move model to GPU/CPU

            models.append(model)
            print(f"  ✓ Fold {fold} model loaded")

        print(f"✓ Successfully loaded {len(models)} models\n")
        return models

    @staticmethod
    def _remove_dataparallel_prefix(state_dict: dict) -> dict:
        """
        Remove the 'module.' prefix from keys in a state_dict saved with DataParallel.

        Args:
            state_dict: Model weight dictionary

        Returns:
            Weight dictionary with the prefix removed

        Why not use try-except with a direct load_state_dict call?
            → Explicitly handling the prefix presence improves readability.
        """
        if not any(k.startswith('module.') for k in state_dict.keys()):
            return state_dict  # Return as is if no prefix is found

        # Create a new dictionary with modified keys
        new_state_dict = OrderedDict()
        for key, value in state_dict.items():
            new_key = key.replace('module.', '')
            new_state_dict[new_key] = value

        return new_state_dict


# ============================================================================
# Inference Engine
# ============================================================================

class InferenceEngine:
    """
    Engine for executing TTA + Ensemble inference.
    """

    def __init__(
        self,
        models: list[nn.Module],
        config: InferenceConfig
    ):
        """
        Args:
            models: List of trained models (for 5 folds)
            config: Configuration object
        """
        self.models = models
        self.config = config

    def predict_single_view(
        self,
        loader: DataLoader
    ) -> dict[str, np.ndarray]:
        """
        Predict with 5-Fold Ensemble for one TTA view.

        Args:
            loader: DataLoader (with a specific TTA transform applied)

        Returns:
            Dictionary of predictions in the format {'total': [N], 'gdm': [N], 'green': [N]}
        """
        view_preds = {'total': [], 'gdm': [], 'green': []}

        with torch.no_grad(): # Disable gradient calculation
            for img_left, img_right in tqdm(loader, desc="  Predicting", leave=False):
                img_left = img_left.to(self.config.device)
                img_right = img_right.to(self.config.device)

                # Collect predictions from 5 folds
                fold_preds = {'total': [], 'gdm': [], 'green': []}

                for model in self.models:
                    pred_total, pred_gdm, pred_green = model(img_left, img_right)
                    fold_preds['total'].append(pred_total.cpu())
                    fold_preds['gdm'].append(pred_gdm.cpu())
                    fold_preds['green'].append(pred_green.cpu())

                # Average predictions across 5 folds
                avg_total = torch.mean(torch.stack(fold_preds['total']), dim=0)
                avg_gdm = torch.mean(torch.stack(fold_preds['gdm']), dim=0)
                avg_green = torch.mean(torch.stack(fold_preds['green']), dim=0)

                view_preds['total'].append(avg_total.numpy())
                view_preds['gdm'].append(avg_gdm.numpy())
                view_preds['green'].append(avg_green.numpy())

        # Concatenate results from all batches
        return {
            k: np.concatenate(v).flatten()
            for k, v in view_preds.items()
        }

    def predict_with_tta(
        self,
        test_df: pd.DataFrame,
        tta_transforms: list[A.Compose]
    ) -> dict[str, np.ndarray]:
        """
        Execute final prediction with TTA + Ensemble.

        Args:
            test_df: Test data DataFrame
            tta_transforms: List of transforms for TTA

        Returns:
            Dictionary of final predictions after TTA averaging
        """
        print(f"\nStarting TTA inference: {len(tta_transforms)} Views × {self.config.n_folds} Folds")

        all_view_preds: list[dict[str, np.ndarray]] = []

        for i, transform in enumerate(tta_transforms):
            print(f"--- TTA View {i+1}/{len(tta_transforms)} ---")

            # Create a dedicated Dataset and DataLoader for this view
            dataset = TestBiomassDataset(
                test_df,
                transform,
                self.config.test_image_dir
            )

            loader = DataLoader(
                dataset,
                batch_size=self.config.batch_size,
                shuffle=False,
                num_workers=self.config.num_workers,
                pin_memory=True
            )

            # Perform 5-Fold Ensemble prediction
            view_preds = self.predict_single_view(loader)
            all_view_preds.append(view_preds)

            print(f"  ✓ View {i+1} completed")

        # TTA Ensemble (average across all views)
        print("\nCalculating TTA Ensemble (averaging all views)...")
        final_preds = {
            'total': np.mean([p['total'] for p in all_view_preds], axis=0),
            'gdm': np.mean([p['gdm'] for p in all_view_preds], axis=0),
            'green': np.mean([p['green'] for p in all_view_preds], axis=0)
        }

        print("✓ Inference completed\n")
        return final_preds


# ============================================================================
# Submission Creation
# ============================================================================

class SubmissionCreator:
    """
    Class for creating the Kaggle submission CSV from predictions.
    """

    def __init__(self, config: InferenceConfig):
        """
        Args:
            config: Configuration object
        """
        self.config = config

    def _build_wide_predictions(
        self,
        predictions: dict[str, np.ndarray],
        df_unique: pd.DataFrame
    ) -> pd.DataFrame:
        """Return a per-image wide DataFrame of all five targets."""
        # 1. Get the 3 predictions output by the model
        pred_total = predictions['total']
        pred_gdm = predictions['gdm']
        pred_green = predictions['green']

        # 2. Calculate the remaining 2 targets using relationships (clip negative values to 0)
        pred_clover = np.maximum(0, pred_gdm - pred_green)
        pred_dead = np.maximum(0, pred_total - pred_gdm)

        # 3. Create a wide-format DataFrame
        preds_wide = pd.DataFrame({
            'image_path': df_unique['image_path'],
            'Dry_Green_g': pred_green,
            'Dry_Dead_g': pred_dead,
            'Dry_Clover_g': pred_clover,
            'GDM_g': pred_gdm,
            'Dry_Total_g': pred_total
        })

        preds_wide['image_id'] = preds_wide['image_path'].apply(
            lambda p: Path(p).stem
        )
        return preds_wide

    def create(
        self,
        predictions: dict[str, np.ndarray],
        test_df_long: pd.DataFrame,
        test_df_unique: pd.DataFrame
    ) -> None:
        """
        Create and save the submission CSV from predictions.
        """
        print("Creating submission CSV...")
        preds_wide = self._build_wide_predictions(predictions, test_df_unique)

        # 4. Convert to long format (unpivot)
        preds_long = preds_wide.melt(
            id_vars=['image_path', 'image_id'],
            value_vars=self.config.all_target_cols,
            var_name='target_name',
            value_name='target'
        )

        # 5. Merge with the original test.csv to get sample_id
        submission = pd.merge(
            test_df_long[['sample_id', 'image_path', 'target_name']],
            preds_long,
            on=['image_path', 'target_name'],
            how='left'
        )

        # 6. Format and save
        submission = submission[['sample_id', 'target']]
        submission.to_csv(self.config.submission_file, index=False)

        print(f"\n🎉 Submission saved to: {self.config.submission_file}")
        print("\n--- First 5 rows ---")
        print(submission.head())
        print("\n--- Last 5 rows ---")
        print(submission.tail())

    def save_oof(
        self,
        predictions: dict[str, np.ndarray],
        df_unique: pd.DataFrame,
        output_path: str
    ) -> None:
        """Persist per-image predictions for stacking/weight search."""
        preds_wide = self._build_wide_predictions(predictions, df_unique)
        cols = ['image_id', 'image_path'] + self.config.all_target_cols
        preds_wide[cols].to_csv(output_path, index=False)
        print(f"OOF-style predictions saved to {output_path}")


# ============================================================================
# Inference Pipeline
# ============================================================================

class InferencePipeline:
    """
    Class that orchestrates the entire inference pipeline.
    """

    def __init__(self, config: InferenceConfig):
        """
        Args:
            config: Configuration object
        """
        self.config = config
        self.model_loader = ModelLoader(config)
        self.tta_factory = TTATransformFactory(config.img_size)
        self.submission_creator = SubmissionCreator(config)

    def run(
        self,
        split: str = 'test',
        save_oof: bool = False,
        oof_output: str = 'oof_model2.csv'
    ) -> None:
        """
        Execute the entire inference pipeline.

        Processing flow:
        1. Load test data
        2. Load models (5-Fold)
        3. Run TTA inference (3 Views × 5 Folds)
        4. Create submission file
        """
        split = split.lower()
        csv_path, image_dir = self.config.resolve_split_paths(split)

        print(f"\n{'='*70}")
        print(f"🚀 Starting Inference Pipeline ({split} split)")
        print(f"{'='*70}")

        models: list[nn.Module] | None = None
        engine: Optional[InferenceEngine] = None
        predictions: Optional[dict[str, np.ndarray]] = None

        try:
            # 1. Load split data
            split_df_long, split_df_unique = self._load_split_data(csv_path)
            self.config.test_image_dir = image_dir

            # 2. Load models
            models = self.model_loader.load_fold_models()

            # 3. Run TTA inference
            engine = InferenceEngine(models, self.config)
            tta_transforms = self.tta_factory.get_tta_transforms()
            predictions = engine.predict_with_tta(split_df_unique, tta_transforms)

            # 4. Persist predictions
            if split == 'test':
                self.submission_creator.create(
                    predictions,
                    split_df_long,
                    split_df_unique
                )
            elif save_oof:
                self.submission_creator.save_oof(
                    predictions,
                    split_df_unique,
                    oof_output
                )
            else:
                print("Train split inference finished (no OOF file requested).")

            print("\n✨ Inference Pipeline Completed Successfully ✨")

        except Exception as e:
            print(f"\n❌ An error occurred: {e}")
            raise

        finally:
            # Free up memory
            if models is not None:
                del models
            if engine is not None:
                del engine
            if predictions is not None:
                del predictions
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    def _load_split_data(self, csv_path: Path) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        Load either train or test CSV in long and unique formats.
        """
        print(f"\nLoading data split: {csv_path}")

        if not csv_path.exists():
            raise FileNotFoundError(f"Split CSV not found: {csv_path}")

        split_df_long = pd.read_csv(csv_path)
        split_df_unique = split_df_long.drop_duplicates(
            subset=['image_path']
        ).reset_index(drop=True)

        print(f"  Long format data: {len(split_df_long)} rows")
        print(f"  Unique images: {len(split_df_unique)} images\n")

        return split_df_long, split_df_unique


# ============================================================================
# Main Execution Block
# ============================================================================

if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description="ConvNeXt inference + optional OOF generation"
    )
    parser.add_argument(
        "--split",
        choices=["test", "train"],
        default="test",
        help="Dataset split to run inference on."
    )
    parser.add_argument(
        "--save-oof",
        action="store_true",
        help="Persist per-image predictions when running on the train split."
    )
    parser.add_argument(
        "--skip-train-oof",
        action="store_true",
        help="Skip the automatic train pass that creates oof_model2.csv when running test inference."
    )
    parser.add_argument(
        "--oof-output",
        default="oof_model2.csv",
        help="Output filename for train-split predictions."
    )
    args = parser.parse_args()

    config = InferenceConfig()
    config.display_info()

    pipeline = InferencePipeline(config)
    if args.split == "test":
        if args.skip_train_oof:
            print("\n⚠️  Skipping train-split OOF generation per --skip-train-oof.")
        else:
            pipeline.run(
                split="train",
                save_oof=True,
                oof_output=args.oof_output
            )
        pipeline.run(
            split="test",
            save_oof=False,
            oof_output=args.oof_output
        )
    else:
        pipeline.run(
            split=args.split,
            save_oof=args.save_oof,
            oof_output=args.oof_output
        )

    print("\n" + "="*70)
    print("🎊 All inference processes have completed!")
    print("="*70)


## Model 3 – SigLIP + Gradient Boosting (Paper alignment)

This block follows Wada's fusion of SigLIP and CoAtNet embeddings to cover the high intra-class variance noted in the CSIRO paper (multi-season, multi-location imagery). Feeding those embeddings into boosted regressors gives a flexible way to respect the log-scaled target behavior and the dominance of Dry_Total_g in the final score, which the paper formalizes via weighted R² (`CSIRO paper, 2510.22916v1.txt:355-382`). As with the other models, only image paths enter inference, matching the paper's constraint that NDVI/height metadata be confined to training-only feature engineering (`CSIRO paper, 2510.22916v1.txt:273-343`).


In [ ]:
%%writefile model_3_siglip.py
# ==============================================================================
# Model 3: SigLIP + CoAtNet feature ensemble with boosted heads + OOF export
# ==============================================================================
import os
import warnings
from copy import deepcopy
from pathlib import Path

import catboost
import numpy as np
import polars as pl
try:
    import timm
except ImportError:
    timm = None
import torch
import torchvision.transforms as T
from PIL import Image
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GroupKFold
from tqdm.auto import tqdm
from transformers import AutoImageProcessor, AutoModel

warnings.filterwarnings("ignore")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

DATA_PATH = Path("/kaggle/input/csiro-biomass")
SIGLIP_PATH = "/kaggle/input/google-siglip-so400m-patch14-384/transformers/default/1/"
TARGETS = [
    "Dry_Clover_g",
    "Dry_Dead_g",
    "Dry_Green_g",
    "Dry_Total_g",
    "GDM_g",
]
BACKBONES = [
    b.strip().lower()
    for b in os.environ.get("MODEL3_BACKBONES", "siglip,coatnet").split(",")
    if b.strip()
]
if not BACKBONES:
    BACKBONES = ["siglip", "coatnet"]


class FeatureExtractor:
    """Unified interface for SigLIP and CoAtNet feature extraction."""

    def __init__(self, name: str):
        self.name = name
        if name == "siglip":
            self.batch_size = 20
            self.processor = AutoImageProcessor.from_pretrained(SIGLIP_PATH)
            self.model = AutoModel.from_pretrained(SIGLIP_PATH).to(device)
            self.model.eval()
        elif name == "coatnet":
            if timm is None:
                raise RuntimeError("timm is not installed; cannot use CoAtNet backbone.")
            self.batch_size = 24
            model_name = os.environ.get("MODEL3_COATNET_NAME", "coatnet_0_rw_224.sw_in12k_ft_in1k")
            try:
                self.model = timm.create_model(
                    model_name,
                    pretrained=True,
                    num_classes=0,
                    drop_rate=0.0,
                    global_pool="avg",
                ).to(device)
            except RuntimeError as err:
                raise RuntimeError(
                    f"Unable to instantiate CoAtNet '{model_name}'. "
                    "Specify a valid TIMM model via MODEL3_COATNET_NAME "
                    "or set MODEL3_BACKBONES=siglip."
                ) from err
            self.model.eval()
            self.transform = T.Compose(
                [
                    T.Resize((384, 384)),
                    T.ToTensor(),
                    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ]
            )
        else:
            raise ValueError(f"Unsupported backbone: {name}")

    def encode(self, image_paths: list[str], desc: str) -> np.ndarray:
        """Extract embeddings for all provided image paths."""
        features: list[np.ndarray] = []
        for start in tqdm(range(0, len(image_paths), self.batch_size), desc=desc):
            batch_paths = image_paths[start : start + self.batch_size]

            if self.name == "siglip":
                pil_batch = [
                    Image.open(DATA_PATH / path).convert("RGB") for path in batch_paths
                ]
                inputs = self.processor(images=pil_batch, return_tensors="pt").to(device)
                with torch.no_grad():
                    feats = self.model.get_image_features(**inputs)
                features.append(feats.cpu().numpy())
                for img in pil_batch:
                    img.close()
            else:
                tensors = []
                for path in batch_paths:
                    img = Image.open(DATA_PATH / path).convert("RGB")
                    tensors.append(self.transform(img))
                    img.close()
                batch = torch.stack(tensors).to(device)
                with torch.no_grad():
                    feats = self.model(batch)
                features.append(feats.cpu().numpy())

        return np.concatenate(features, axis=0).astype(np.float32)


def competition_metric(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    weights = np.array([0.1, 0.1, 0.1, 0.5, 0.2])
    y_weighted_mean = np.average(y_true, weights=weights, axis=1).mean()
    ss_res = np.average((y_true - y_pred) ** 2, weights=weights, axis=1).mean()
    ss_tot = np.average(
        (y_true - y_weighted_mean) ** 2, weights=weights, axis=1
    ).mean()
    return 1 - ss_res / ss_tot


def cross_validate(model, data, data_test, x_columns) -> tuple[np.ndarray, np.ndarray]:
    X = data.select(x_columns).to_numpy()
    X_test = data_test.select(x_columns).to_numpy()
    y_true = data.select(TARGETS).to_numpy()

    y_pred_oof = np.zeros_like(y_true)
    y_pred_test = np.zeros((len(X_test), len(TARGETS)))

    kf = GroupKFold(n_splits=5)
    groups = data.select("group")

    for i, (train_index, val_index) in enumerate(kf.split(X, groups=groups)):
        for label_idx in range(len(TARGETS)):
            estimator = deepcopy(model)
            estimator.fit(X[train_index], y_true[train_index, label_idx])
            y_pred_oof[val_index, label_idx] = estimator.predict(X[val_index]).clip(0)
            y_pred_test[:, label_idx] += (
                estimator.predict(X_test).clip(0) / kf.n_splits
            )

        score = competition_metric(y_true[val_index], y_pred_oof[val_index])
        print(f"Fold {i}: Score = {score:.6f}")

    full_cv_score = competition_metric(y_true, y_pred_oof)
    print(f"Full CV Score: {full_cv_score:.6f}")
    return y_pred_oof, y_pred_test


def main():
    print("Processing training data...")
    train = pl.read_csv(DATA_PATH / "train.csv")
    df = (
        train.with_columns(
            [
                pl.when(pl.col("target_name") == label)
                .then(pl.col("target"))
                .alias(label)
                for label in TARGETS
            ]
        )
        .group_by("image_path")
        .agg(
            [pl.col(label).mean() for label in TARGETS]
            + [
                pl.concat_str(["Sampling_Date", "State"], separator=" ")
                .alias("group")
                .first()
            ]
        )
        .sort("image_path")
    )

    print("Processing test data...")
    test = pl.read_csv(DATA_PATH / "test.csv")
    df_test = (
        test.group_by("image_path")
        .len()
        .with_columns(pl.col("image_path"))
        .sort("image_path")
    )

    train_paths = df.select("image_path").to_series().to_list()
    test_paths = df_test.select("image_path").to_series().to_list()

    feature_frames_train = []
    feature_frames_test = []
    feature_columns: list[str] = []

    enabled_backbones: list[str] = []
    for backbone in BACKBONES:
        try:
            extractor = FeatureExtractor(backbone)
        except Exception as exc:
            print(f"[Warning] Skipping backbone '{backbone}': {exc}")
            continue

        print(f"\nExtracting {backbone} features...")
        train_feats = extractor.encode(train_paths, f"{backbone} train")
        test_feats = extractor.encode(test_paths, f"{backbone} test")
        cols = [f"{backbone}_f{i:04d}" for i in range(train_feats.shape[1])]
        feature_frames_train.append(pl.DataFrame(train_feats, schema=cols))
        feature_frames_test.append(pl.DataFrame(test_feats, schema=cols))
        feature_columns.extend(cols)
        enabled_backbones.append(backbone)

        del extractor
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    if not enabled_backbones:
        raise RuntimeError(
            "No valid backbones were initialized. "
            "Set MODEL3_BACKBONES to include at least 'siglip'."
        )

    df_aug = pl.concat([df] + feature_frames_train, how="horizontal")
    df_test_aug = pl.concat([df_test] + feature_frames_test, how="horizontal")

    print("\n--- [Comparison] DummyRegressor ---")
    cross_validate(DummyRegressor(), df_aug, df_test_aug, feature_columns)

    print("\n--- [Comparison] Ridge ---")
    cross_validate(Ridge(), df_aug, df_test_aug, feature_columns)

    print("\n--- [Comparison] Lasso ---")
    cross_validate(Lasso(), df_aug, df_test_aug, feature_columns)

    print("\n--- [Final Model] GradientBoostingRegressor ---")
    oof_pred_gb, pred_test_gb = cross_validate(
        GradientBoostingRegressor(random_state=42),
        df_aug,
        df_test_aug,
        feature_columns,
    )

    print("\n--- [Final Model] CatBoostRegressor ---")
    oof_pred_cb, pred_test_cb = cross_validate(
        catboost.CatBoostRegressor(verbose=False, iterations=200, random_state=42),
        df_aug,
        df_test_aug,
        feature_columns,
    )

    print("\nEnsembling and saving OOF predictions...")
    oof_pred_ensemble = (oof_pred_gb + oof_pred_cb) / 2
    oof_df = df_aug.select(["image_path"]).to_pandas()
    oof_df[TARGETS] = oof_pred_ensemble
    oof_df.to_csv("oof_model3.csv", index=False)

    print("\nEnsembling the predictions of the two models...")
    pred_test = (pred_test_gb + pred_test_cb) / 2

    pred_with_id = pl.concat(
        [df_test.select("image_path"), pl.DataFrame(pred_test, schema=TARGETS)],
        how="horizontal",
    )

    pred_save = (
        test.join(pred_with_id, on="image_path")
        .with_columns(
            pl.coalesce(
                *[
                    pl.when(pl.col("target_name") == col).then(pl.col(col))
                    for col in TARGETS
                ]
            ).alias("target")
        )
        .select("sample_id", "target")
    )
    pred_save.write_csv("submission_SigLIP.csv")
    print("\nCreated submission_SigLIP.csv.")
    print("Partial submission file:")
    print(pred_save.head())


if __name__ == "__main__":
    main()


## Inference (Paper alignment)

The orchestration cell now documents why we keep the exact Kaggle directory conventions from Wada's notebook while ensuring the runtime only consumes imagery. By running the ConvNeXt pipeline on both train (to produce OOF traces) and test splits we can reproduce the paper's recommendation to validate model behavior across locations/dates before deploying (`CSIRO paper, 2510.22916v1.txt:273-343`). The inference loop also mirrors the weighted scoring procedure by predicting the three primary heads first, then reconstructing GDM and Dry_Total to maintain the biomass mass-balance highlighted in the dataset description (`CSIRO paper, 2510.22916v1.txt:260-349`).


In [ ]:
# import time 

!python /kaggle/working/model_1_dino_giant.py
# time.sleep(10)
!python /kaggle/working/model_2_ConvnextTiny.py
# time.sleep(10)
!python /kaggle/working/model_3_siglip.py

## Weighted Average + Stacking Ensemble (Paper alignment)

This final stage matches Wada's ensemble search but grounds every decision in the paper's weighted R² metric that prioritizes Dry_Total_g (weight 0.5) while retaining fidelity to Dry_Green_g, Dry_Dead_g, Dry_Clover_g, and GDM_g (`CSIRO paper, 2510.22916v1.txt:355-382`). The post-processing constraints (non-negative components, components summing to totals) explicitly enforce the biomass conservation rules described in the dataset section (`CSIRO paper, 2510.22916v1.txt:260-349`), so the convex weight search plus Ridge stacker both move the predictions toward the notion of optimality defined in the paper.


In [ ]:
# ====================================================================================
# Advanced Ensemble: weight search + stacking + post-processing
# ====================================================================================

from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

DATA_ROOT = Path("/kaggle/input/csiro-biomass")
TARGET_COLS = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]
MODEL_CONFIG = {
    "dino_lgbm": {
        "submission": Path("submission_dino_giant.csv"),
        "oof": Path("oof_model1.csv"),
    },
    "convnext_tta": {
        "submission": Path("submission_ConvnextTiny.csv"),
        "oof": Path("oof_model2.csv"),
    },
    "siglip_coatnet": {
        "submission": Path("submission_SigLIP.csv"),
        "oof": Path("oof_model3.csv"),
    },
}
WEIGHT_SEARCH_STEP = 0.05


def competition_metric(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    weights = np.array([0.1, 0.1, 0.1, 0.5, 0.2])
    y_weighted_mean = np.average(y_true, weights=weights, axis=1).mean()
    ss_res = np.average((y_true - y_pred) ** 2, weights=weights, axis=1).mean()
    ss_tot = np.average(
        (y_true - y_weighted_mean) ** 2, weights=weights, axis=1
    ).mean()
    return 1 - ss_res / ss_tot


def ensure_file(path: Path) -> Path:
    if not path.exists():
        raise FileNotFoundError(f"Required file not found: {path}")
    return path


def load_train_targets() -> pd.DataFrame:
    train = pd.read_csv(DATA_ROOT / "train.csv")
    train["image_id"] = train["sample_id"].str.split("__").str[0]
    train["target_name"] = train["sample_id"].str.split("__").str[1]
    return (
        train.pivot_table(
            index="image_id",
            columns="target_name",
            values="target",
            aggfunc="mean",
        )
        .reindex(columns=TARGET_COLS)
        .sort_index()
    )


def load_submission_wide(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    if df.empty:
        raise ValueError(f"{path} is empty.")
    tmp = df.copy()
    tmp[["image_id", "target_name"]] = tmp["sample_id"].str.split("__", expand=True)
    wide = (
        tmp.pivot_table(
            index="image_id",
            columns="target_name",
            values="target",
            aggfunc="mean",
        )
        .reindex(columns=TARGET_COLS)
        .fillna(0)
        .sort_index()
    )
    wide.index.name = "image_id"
    return wide


def load_oof_frame(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    if "image_id" not in df.columns:
        if "image_path" in df.columns:
            df["image_id"] = df["image_path"].apply(lambda p: Path(p).stem)
        elif "sample_id" in df.columns:
            df["image_id"] = df["sample_id"].str.split("__").str[0]
        else:
            raise ValueError(f"Cannot infer image_id column for {path}")
    frame = (
        df[["image_id"] + TARGET_COLS]
        .drop_duplicates("image_id")
        .set_index("image_id")
        .reindex(columns=TARGET_COLS)
        .sort_index()
    )
    return frame


def collect_frames(kind: str) -> dict[str, pd.DataFrame]:
    frames = {}
    for name, cfg in MODEL_CONFIG.items():
        path = ensure_file(cfg[kind])
        frames[name] = (
            load_oof_frame(path)
            if kind == "oof"
            else load_submission_wide(path)
        )
    return frames


def align_training_data(
    targets: pd.DataFrame, oof_frames: dict[str, pd.DataFrame]
) -> tuple[pd.DataFrame, dict[str, pd.DataFrame]]:
    common_index = targets.index
    for frame in oof_frames.values():
        common_index = common_index.intersection(frame.index)
    targets = targets.loc[common_index].sort_index()
    aligned_oof = {k: v.loc[common_index].sort_index() for k, v in oof_frames.items()}
    return targets, aligned_oof


def optimize_weights(
    oof_frames: dict[str, pd.DataFrame], targets: pd.DataFrame, step: float
) -> tuple[dict[str, float], float]:
    model_names = list(oof_frames.keys())
    mats = np.stack([oof_frames[name].to_numpy() for name in model_names], axis=0)
    y_true = targets.to_numpy()

    best_score = -np.inf
    best_weights = None
    grid = np.arange(0.0, 1.0 + 1e-9, step)

    for w1 in grid:
        for w2 in grid:
            w3 = 1.0 - w1 - w2
            if w3 < -1e-9:
                continue
            weights = np.array([w1, w2, w3])
            if np.any(weights < 0):
                continue
            blended = np.tensordot(weights, mats, axes=(0, 0))
            score = competition_metric(y_true, blended)
            if score > best_score:
                best_score = score
                best_weights = weights

    weight_dict = dict(zip(model_names, best_weights))
    return weight_dict, best_score


def build_meta_features(
    frames: dict[str, pd.DataFrame], index: pd.Index
) -> pd.DataFrame:
    parts = []
    for name, frame in frames.items():
        aligned = frame.reindex(index).fillna(0)
        parts.append(aligned.add_prefix(f"{name}_"))
    return pd.concat(parts, axis=1)


def train_stacker(
    oof_frames: dict[str, pd.DataFrame],
    targets: pd.DataFrame,
    test_frames: dict[str, pd.DataFrame],
) -> tuple[pd.DataFrame, float]:
    train_features = build_meta_features(oof_frames, targets.index)

    test_index = None
    for frame in test_frames.values():
        test_index = frame.index if test_index is None else test_index.intersection(frame.index)
    if test_index is None:
        raise ValueError("No test predictions found.")
    test_features = build_meta_features(test_frames, test_index)

    X = train_features.to_numpy()
    y = targets.to_numpy()
    X_test = test_features.to_numpy()

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof_pred = np.zeros_like(y)
    test_pred = np.zeros((len(test_features), len(TARGET_COLS)))

    for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
        print(f"[Stacker] Fold {fold}")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        for target_idx in range(len(TARGET_COLS)):
            ridge = Ridge(alpha=1.0)
            ridge.fit(X_train, y_train[:, target_idx])
            oof_pred[val_idx, target_idx] = ridge.predict(X_val)
            test_pred[:, target_idx] += ridge.predict(X_test) / kf.n_splits

    stack_score = competition_metric(y, oof_pred)
    stacked_df = pd.DataFrame(
        test_pred, columns=TARGET_COLS, index=test_index
    ).sort_index()
    return stacked_df, stack_score


def apply_constraints(wide_df: pd.DataFrame) -> pd.DataFrame:
    df = wide_df.copy()
    comp_cols = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g"]
    enforce_cols = comp_cols + ["Dry_Total_g", "GDM_g"]
    df[enforce_cols] = df[enforce_cols].clip(lower=0)

    comp_sum = df[comp_cols].sum(axis=1)
    safe_sum = comp_sum.replace(0, np.nan)
    ratio = (df["Dry_Total_g"] / safe_sum).clip(0.8, 1.2)
    idx = ratio.notna()
    df.loc[idx, comp_cols] = df.loc[idx, comp_cols].mul(ratio[idx], axis=0)
    df["Dry_Total_g"] = df[comp_cols].sum(axis=1)
    df["GDM_g"] = np.maximum(
        df["GDM_g"], df["Dry_Green_g"] + df["Dry_Clover_g"]
    )
    df = df.fillna(0)
    return df


def wide_to_submission(wide_df: pd.DataFrame, fname: str) -> pd.DataFrame:
    long_df = (
        wide_df.reset_index()
        .melt(id_vars="image_id", var_name="target_name", value_name="target")
    )
    long_df["sample_id"] = long_df["image_id"] + "__" + long_df["target_name"]
    submission = long_df[["sample_id", "target"]].sort_values("sample_id")
    submission.to_csv(fname, index=False)
    return submission


def main():
    print("Loading OOF and submission files...")
    oof_frames = collect_frames("oof")
    test_frames = collect_frames("submission")
    train_targets = load_train_targets()
    train_targets, oof_frames = align_training_data(train_targets, oof_frames)

    print("\nSearching for optimal convex weights...")
    best_weights, weight_score = optimize_weights(
        oof_frames, train_targets, WEIGHT_SEARCH_STEP
    )
    print(f"Best weights: {best_weights} (OOF weighted R2={weight_score:.5f})")

    weighted_wide = sum(
        best_weights[name] * test_frames[name] for name in test_frames.keys()
    )
    weighted_wide = apply_constraints(weighted_wide)
    weighted_wide.index.name = "image_id"
    weighted_submission = wide_to_submission(weighted_wide, "submission_weighted.csv")
    print(f"Saved weight-searched blend to submission_weighted.csv ({len(weighted_submission)} rows)")

    print("\nTraining Ridge stacker on OOF features...")
    stacked_wide, stack_score = train_stacker(oof_frames, train_targets, test_frames)
    print(f"[Stacker] OOF weighted R2: {stack_score:.5f}")
    stacked_wide = apply_constraints(stacked_wide)
    stacked_wide.index.name = "image_id"
    final_submission = wide_to_submission(stacked_wide, "submission.csv")
    print(f"\n🎉 Final stacked submission written to submission.csv ({len(final_submission)} rows)")


if __name__ == "__main__":
    main()
